In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, cm
from mpl_toolkits.mplot3d import Axes3D  

In [ ]:
#constants
nx = 20
ny = 20
nt = 100
dx = 2 / nx
dy = 2 / ny
xs = np.linspace(0, 2, nx)
ys = np.linspace(0, 2, ny)
X, Y = np.meshgrid(xs, ys)
F = 1 

rho = 1
nu =0.2
dt = 0.01

#init 
u = np.zeros((ny, nx))
v = np.zeros((ny, nx))
p = np.zeros((ny, nx)) 
b = np.zeros((ny, nx))

In [ ]:
def create_B(b, rho, dt, u, v, dx, dy):
    
    b[1:-1, 1:-1] = (rho * (1 / dt * 
                    ((u[1:-1, 2:] - u[1:-1, 0:-2]) / (2 * dx) + (v[2:, 1:-1] - v[0:-2, 1:-1]) / (2 * dy)) -
                    ((u[1:-1, 2:] - u[1:-1, 0:-2]) / (2 * dx))**2 -
                      2 * ((u[2:, 1:-1] - u[0:-2, 1:-1]) / (2 * dy) *
                           (v[1:-1, 2:] - v[1:-1, 0:-2]) / (2 * dx))-
                          ((v[2:, 1:-1] - v[0:-2, 1:-1]) / (2 * dy))**2))

#BC x = 2 periodic
    b[1:-1,-1] = (rho * (1 / dt * 
                    ((u[1:-1, 0] - u[1:-1, -2]) / (2 * dx) + (v[2:, -1] - v[0:-2, -1]) / (2 * dy)) -
                    ((u[1:-1, 0] - u[1:-1, -2]) / (2 * dx))**2 -
                      2 * ((u[2:, -1] - u[0:-2, -1]) / (2 * dy) *
                           (v[1:-1, 0] - v[1:-1, -2]) / (2 * dx))-
                          ((v[2:, -1] - v[0:-2, -1]) / (2 * dy))**2))
#BC x = 0 periodic
    b[1:-1,-1] = (rho * (1 / dt * 
                    ((u[1:-1, 1] - u[1:-1, -1]) / (2 * dx) + (v[2:, 0] - v[0:-2, 0]) / (2 * dy)) -
                    ((u[1:-1, 1] - u[1:-1, -1]) / (2 * dx))**2 -
                      2 * ((u[2:, 0] - u[0:-2, 0]) / (2 * dy) *
                           (v[1:-1, -1] - v[1:-1, 1]) / (2 * dx))-
                          ((v[2:, 0] - v[0:-2, 0]) / (2 * dy))**2))

    return b

def pressure_poisson(p, dx, dy, b):
    p0 = p.copy()
    
    for _ in range(50):
        p0 = p.copy()
        p[1:-1, 1:-1] = (((p0[1:-1, 2:] + p0[1:-1, 0:-2]) * dy**2 + 
                          (p0[2:, 1:-1] + p0[0:-2, 1:-1]) * dx**2) /
                          (2 * (dx**2 + dy**2)) -
                          dx**2 * dy**2 / (2 * (dx**2 + dy**2)) * 
                          b[1:-1,1:-1])

#BC x = 2 periodic
        p[1:-1, -1] = (((p0[1:-1, 0] + p0[1:-1, -2]) * dy**2 + 
                          (p0[2:, -1] + p0[0:-2, -1]) * dx**2) /
                          (2 * (dx**2 + dy**2)) -
                          dx**2 * dy**2 / (2 * (dx**2 + dy**2)) * 
                          b[1:-1,-1])
#BC x = 0 periodic
        p[1:-1, 0] = (((p0[1:-1, 1] + p0[1:-1, -1]) * dy**2 + 
                          (p0[2:, 0] + p0[0:-2, 0]) * dx**2) /
                          (2 * (dx**2 + dy**2)) -
                          dx**2 * dy**2 / (2 * (dx**2 + dy**2)) * 
                          b[1:-1,0])
#BC wall dp/dy = 0 at y=0
        p[-1,:] = p[-2,:]
        p[0,:] = p[1,:]

    return p